In [ ]:
%cd ../
%matplotlib inline

In [ ]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [ ]:
db_file = 'data/synthetic_data_nofrills_wholebrain.db'
mask_file = '/data/zulqarnain/wholebrain.nii.gz'

HIGH_CONDITION_LAST = 10
OVERWRITE = True

existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)


In [ ]:
if not existed or OVERWRITE:
    data_files = utils.sorted_glob('/data/zulqarnain/synthetic_data_nofrills_blocked/nifti_files/*.nii')
    regex = re.compile("subject_([0-9]{1,2})_task_([1-3])_stimulus_([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        #         pdb.set_trace()
        [subject, task, stimulus] = m.groups()
        subject = int(subject)
        if int(task)>=0:
            if subject >= 1 and subject <= HIGH_CONDITION_LAST:
                condition = 0
            else:
                condition = 1
            task = int(task)
            stimulus = int(stimulus)
            logging.info("Recording blocks in %s", data_file)

            synthetic_block = niidb.FMriActivationBlock(zscore=False)
            if task == 1:
                synthetic_block.task = 'rest_' + str(stimulus)
            elif task == 2:
                synthetic_block.task = 'baseline_' + str(stimulus)
            else:
                synthetic_block.task = 'task_' + str(stimulus)
            synthetic_block.filename = data_file
            synthetic_block.subject = subject
            synthetic_block.start_time = 0
            synthetic_block.end_time = None
            synthetic_block.individual_differences = {
                'condition': condition,
                'stimulus_group': task,
            }
            db.upsert(synthetic_block)
            logging.info('Subject %d performing task %s', subject, synthetic_block.task)


In [ ]:
logging.info('Finished building NiiDb out of synthetic dataset')